In [46]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [47]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules import loss
import torch.optim as optim

In [48]:
class LSTMClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, tagset_size, batch_size):
        # 親クラスのコンストラクタ。決まり文句
        super(LSTMClassifier, self).__init__()
        # 隠れ層の次元数。これは好きな値に設定しても行列計算の過程で出力には出てこないので。    
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        # LSTMの隠れ層。これ１つでOK。超便利。
        self.lstm = nn.LSTM(embedding_dim, hidden_dim//2, batch_first=True, bidirectional=True )
        # LSTMの出力を受け取って全結合してsoftmaxに食わせるための１層のネットワーク
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        # softmaxのLog版。dim=0で列、dim=1で行方向を確率変換。
        # self.softmax = 
    
    def forward(self, x):
        #embeds.size() = (batch_size × len(sentence) × embedding_dim)
        batch_size, seq_len = x.shape[0], x.shape[1]
        _, hidden_layer = self.lstm(x)
        # print(hidden_layer)
        bilstm_out = torch.cat([hidden_layer[0][0], hidden_layer[0][1]], dim=1)
        # y = self.hidden2tag(hidden_layer[0].view(batch_size, -1))

        y = self.hidden2tag(bilstm_out)
        y = F.log_softmax(y, dim=1)
        return y

In [49]:
import pickle
class DataManager:
    def __init__(self, data_path) -> None:
        import os
        import pickle
        self.data_path = data_path
        os.makedirs(data_path, exist_ok=True)
        self.dir = os.listdir(data_path)

    def is_exist(self, name):
        return (name in self.dir)
    
    def save_data(self, name, obj):
        with open(self.data_path+name, "wb") as f:
            pickle.dump(obj, f)
        print("success save : {0}{1}".format(self.data_path, name))

    def load_data(self, name):
        with open(self.data_path+name, "rb") as f:
            obj = pickle.load(f)
        print("success load : {0}{1}".format(self.data_path, name))
        return obj

In [50]:
from pyknp import Juman
from sentence_transformers import SentenceTransformer
import scipy.spatial
Nmodel_path = "/home/yamada/Downloads/training_bert_japanese"
Nmodel = SentenceTransformer(Nmodel_path, show_progress_bar=False)
emb_dim = Nmodel.encode(["お辞儀をしている男性会社員"])[0].shape[0]

In [51]:
def make_X(convs, max_len):
    # emb_dim = nlp("形態素").vector.shape
    X_data = []
    
    for conv in convs :
        # vec_list = np.zeros( (max_len, emb_dim[0]) )
        sentence_vectors = Nmodel.encode(conv)
        # for i, ut in enumerate(conv):
        #     doc = nlp(ut)
        #     vec_list[i] = doc.vector
        X_data.append(sentence_vectors)
    return np.array(X_data)

In [52]:
path = "../hand_labeled/"
datalist = ['DCM', 'DIT', 'IRS']

output = "./"

In [53]:
def read_json_with_NoErr(path:str, datalist:list) -> pd.DataFrame:
    cols = ['did', 'tid', 'usr', 'sys', 'ec']
    df = pd.DataFrame(index=[], columns=cols)

    for p in datalist:
        datapath = Path(path + p + '/')
        for file in datapath.glob("*.json"):
            with open(file, "r") as f:
                json_data = json.load(f)
                did = json_data["did"]
                for t in json_data["turns"]:
                    if t["turn-index"] == 0:
                        continue
                    if t["speaker"] == "U":
                        usr = t["utterance"]
                        continue
                    if t["speaker"] == "S" :
                        tid = t["turn-index"]
                        sys = t["utterance"]
                        if t["error_category"]:
                            ec = t["error_category"]
                        else:
                            ec = ["No-Err"]
                        df = df.append(pd.DataFrame([did, tid, usr, sys, ec], index = cols).T)
    df.reset_index(inplace=True, drop=True)
    return df

In [54]:
df = read_json_with_NoErr(path, datalist)

In [55]:
def extract_continue_convs_with_error(df, length, errors):
    new_convs = []
    continue_conv = []
    did = 0
    for d, u, s, e in zip(df.did, df.usr, df.sys, df.ec):
        # did が変化すれば，別の対話
        if d != did:
            continue_conv = []
            did = d
        continue_conv .append(u)
        continue_conv .append(s)
        for err in errors:
            if len(continue_conv) >= length and err in e:
                new_convs.append( continue_conv[-length:] )
    
    return new_convs

In [56]:
errors = ["Topic transition error", "Unclear intention", "Lack of information"]
# errors = ["Lack of information"]
errors = ["Topic transition error"]
# errors = ["Unclear intention"]
length = 4

In [57]:
UT = extract_continue_convs_with_error(df, length, errors)

In [58]:
import random
UT_NO_ALL = extract_continue_convs_with_error(df, length, [ "No-Err"])
UT_NO = random.sample(UT_NO_ALL, len(UT))
# UT_NO = UT_NO_ALL

In [59]:
print(len(UT))
print(len(UT_NO))

174
174


In [60]:
model_path = "../models/context/"
model_name = "topic4-{0}.pickle".format(length)
modelM = DataManager(model_path)
print(model_name)
if modelM.is_exist(model_name):
    model = modelM.load_data(model_name)
    model.to("cpu")

topic4-4.pickle
success load : ../models/context/topic4-4.pickle


In [17]:
X = make_X(UT + UT_NO, length)
y = np.concatenate( [ np.ones(len(UT)), np.zeros(len(UT_NO)) ] )

In [18]:
with torch.no_grad():
    X_tensor = torch.tensor(X, device='cuda:0').float()
    y_tensor = torch.tensor(y, dtype=torch.long, device='cuda:0')
            # 推論
    y_pred= np.array(model(X_tensor).cpu()).argmax(axis=1)

RuntimeError: Input and parameter tensors are not at the same device, found input tensor at cuda:0 and parameter tensor at cpu

In [ ]:
metrics.accuracy_score(y, y_pred)

0.5948275862068966

In [ ]:
y

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [ ]:
y_pred

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0,

In [ ]:
UT_NO

[['シンガポールと奈良なら、どちらが楽しいでしょうか？',
  '両方楽しかったけど，シンガポールにもう一度行きたい！',
  '私がシンガポールに興味があるのは、一度でいいから、あの有名な株式会社トランスリミットというホテルに宿泊してみたいからです。'],
 ['四季劇場日生劇場で上演中のミュージカル、森永ミルクキャラメルが11月16日に関西最長ロングランを達成したそうです。11月16日時点での通算公演回数は699回との事です。',
  '劇団四季の舞台を一度観てみたいな',
  '劇団四季が2015年5月24日から上演予定の新作のミュージカルはアラジンだそうです。'],
 ['あまり好きではないですが、１日に１度何かは作ってます。',
  'すごいですね。得意な料理とかありますか？',
  'う〜ん、得意というか、豚の生姜焼きとかよく作ります。'],
 ['気持ちがいいですね', '私もそう思います。', 'ですよね'],
 ['では、最近のマイブームは何ですか？', '思いつきません。あなたのマイブームは？', 'マイブームかぁ・・・。旅行です。'],
 ['しりとりは一緒に盛り上がりましょうか？', 'じゃあ私からいきますよ、リンゴ。', 'しりとりを楽しみましょうね'],
 ['隙あらば、次はアメリカ行きたいです。', '私も行きたい', 'あっちの方は謎な遺跡がたくさんあってすごく魅力的ですよね。'],
 ['人とは違うものを身に付けて欲しいと思いますね。', '個性は大事ですね。', 'おやすみ'],
 ['しりとりを楽しみましょうね', 'もう一度いいますよ、リンゴ', '紅玉が良いですね'],
 ['私の場合、カミングアウトバラエティ秘密のケンミンSHOWが好きなので、それをイメージして竹田城跡にとても興味を持ちました。',
  '私は朝の番組で見ました',
  '竹田城跡には行ったことがありますか？ 。'],
 ['ですよねー', '何か話題ある', '最近どうですか？'],
 ['早急に考えるのです', '教えてよ。', '知りたい'],
 ['占いですか！最近は、どんな占いをしましたか？', 'しません', 'では、音楽はどんなのを聴かれていますか？'],
 ['最近はファレル・ウィリアムスですね。', '誰？それ', '神奈川県に住んでい

In [61]:
# real test
leng_c = 4
def make_X_str_y(df, errors):
    X_str = []
    y = []
    y = np.zeros(len(df))
    continue_conv = []
    did = 0
    for i, (d, u, s, e) in enumerate(zip(df.did, df.usr, df.sys, df.ec)):
        # sentence_vectors = Nmodel.encode([u, s])
        if d != did:
            continue_conv = [""]*leng_c
            did = d
        continue_conv.append(u)
        continue_conv.append(s)
        X_str.append( continue_conv[-leng_c:] )
        for err in errors:
            if err in e:
                y[i] = 1

    return X_str, y

In [62]:
X_str, y = make_X_str_y(df, errors)

In [63]:
X = make_X(X_str, length)

In [64]:
with torch.no_grad():
#     X_tensor = torch.tensor(X, device='cuda:0').float()
    X_tensor = torch.tensor(X).float()
    # y_tensor = torch.tensor(y, dtype=torch.long, device='cuda:0')
            # 推論
#     y_pred= np.array(model(X_tensor).cpu()).argmax(axis=1)
    y_pred= np.array(model(X_tensor)).argmax(axis=1)

In [65]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
print('confusion matrix = \n', confusion_matrix(y_true=y, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y, y_pred=y_pred))

print('EM:', metrics.accuracy_score(y, y_pred))
print('F-measure: ', metrics.f1_score(y, y_pred))

confusion matrix = 
 [[1733   75]
 [ 181   11]]
accuracy =  0.872
EM: 0.872
F-measure:  0.07913669064748202


In [66]:
len(y_pred)

2000

In [67]:
metrics.accuracy_score(y, np.zeros(2000))

0.904

In [68]:
y_pred

array([0, 0, 1, ..., 0, 0, 0])

In [72]:
correct_n = 0
bad_n = 0
all_tp_one = 0
for x, t, n in zip(X_str, y, y_pred):
    # t==1 : 本来の破綻
    # n==1 : 予想された破綻
    if t == 1:
        all_tp_one += 1
        if n == 1:
            # t==n==1 : 適切に検出
            correct_n += 1
            # print(x)
        else:
            # 破綻なのに未検出
            bad_n += 1
            print(x)
    else:
        # 破綻ではないのに破綻扱い
        if n == 1:
            bad_n += 1
            # print(x)
print("corrent_n:", correct_n)
print("all_tp_one:", all_tp_one)
print("bad_n:", bad_n)
print("rate c:", correct_n/all_tp_one)

['1000円ぐらいかな？', 'スイカは実がなってたことあります', '意味が分かりません', 'あなたの好きな映画について教えてください']
['焼きそばも好きです', '焼きそばは好きですか。麺が美味しいですね', 'そうですね。スポーツは好きですか？', '麺が美味しいですね']
['はい。麺類は美味しいから好きです。', '麺類は食べやすいですね', '旅行は好きですか？', 'パスタが好きですね']
['婚活パーティーにでも行ってみようかなって思ってます。', 'ケースバイケースで判断するんですね', 'はい、長いこと彼女いないんでいろいろ考えちゃいます。', 'そういえば、よく旅行に行くと言っていましたが、ケースバイケースで臨機応変に対応するのでしょうか？']
['何をもらったら喜ぶ？', '紅玉を求めるかもしれない', 'リンゴの紅玉が好きなんですね。', 'そういえば、この間、旅行したと伺いましたが、奇遇に驚きます']
['暖かくなってきましたね', 'いいですね！', '寒いのは苦手ですか？', 'そうでしたか！\u3000えっと、好きなスポーツは何ですか？']
['', '', '今日は暑いですね。', '読書だそうですが、最近は本を読んでおられますか？']
['ワクワク感がいいよね。', '行くまでが楽しいですね', 'その気持ち凄く分かる。', '今日も買い物には行かれたんですか？']
['急な高波は危ないですね', '高波がすごいですよねえ', '泳ぐのは好きですか？', '読書だそうですが、最近は本を読んでおられますか？']
['こんにちは。今日はいい天気ですね。', '天気がいいとそれだけで元気が出ちゃうよね。', 'こんにちは。今日はいい天気ですね。', 'あ\u3000そうなんですか。\u3000うーんとね、猫を飼ったことはありますか？']
['ところで、今日の昼ご飯は何を食べるつもりですか？', 'インターネットで調べたところ、もしかすると炊き込みご飯です。', 'お昼から炊き込みご飯を作るなんてえらいですね。', '日本では１９８４年に大学同士で接続されたのが始まりなんだ。']
['お昼から炊き込みご飯を作るなんてえらいですね。', '日本では１９８４年に大学同士で接続されたのが始まりなんだ。', 'インターネットはいまではだ